In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import random
from scipy import stats
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)


In [2]:
def concordance_index_scorer(estimator, X, y):
    """
    Computes the concordance index for a survival model's predictions.

    Parameters:
        estimator: Trained survival model with a .predict() method.
        X: Feature matrix for prediction.
        y: Structured array with fields 'event' and 'time'.

    Returns:
        Concordance index (float)
    """
    event, time = y["event"], y["time"]
    prediction = estimator.predict(X)
    return concordance_index_censored(event, time, prediction)[0]

def confidence_interval(cindex_scores, confidence=0.95):
    """
    Computes the mean and confidence interval for a list of C-index scores.

    Parameters:
        cindex_scores: List or array of C-index values.
        confidence: Confidence level (default is 95%).

    Returns:
        Tuple: (mean, lower_bound, upper_bound)
    """
    mean = np.mean(cindex_scores)
    sem = stats.sem(cindex_scores)
    ci = stats.t.interval(confidence, df=len(cindex_scores)-1, loc=mean, scale=sem)
    print(f"Mean C-index: {mean:.2f}, {int(confidence*100)}% CI: ({ci[0]:.2f}, {ci[1]:.2f})")


def clinical_data(train_cohort, test_cohort, clinical_f=None):
    """
    Encodes categorical clinical variables for training and testing datasets.

    Parameters:
    - train_cohort: DataFrame for training data
    - test_cohort: DataFrame for test data
    - clinical_f: Not used, kept for compatibility

    Returns:
    - clinical_train_df: DataFrame with encoded clinical features for training
    - clinical_test_df: DataFrame with encoded clinical features for testing
    """
    
    # Define mappings
    sex_map = {'Female': 1, 'Male': 0}
    smoking_map = {
        'Former smoker': 0,
        'Non smoker': 1,
        'Smoker': 2,
        'Passive smoker': 3
    }
    histo_map = {
        'Carcinoid tumor': 0,
        'Small cell carcinoma': 1,
        'Large cell neuroendocrine carcinoma': 2
    }

    # Apply mappings
    def encode_feature(df, column, mapping):
        return df[column].map(mapping).values.reshape(-1, 1)

    train_sex = encode_feature(train_cohort, 'sex', sex_map)
    test_sex = encode_feature(test_cohort, 'sex', sex_map)

    train_smoking = encode_feature(train_cohort, 'smoking_habit', smoking_map)
    test_smoking = encode_feature(test_cohort, 'smoking_habit', smoking_map)

    train_histo = encode_feature(train_cohort, 'Histological subtype 1_x', histo_map)
    test_histo = encode_feature(test_cohort, 'Histological subtype 1_x', histo_map)

    # Concatenate features
    clinical_train = np.hstack([train_sex, train_smoking, train_histo])
    clinical_test = np.hstack([test_sex, test_smoking, test_histo])

    # Create DataFrames
    columns = ['sex', 'smoking_habit', 'Histological subtype 1_x']
    clinical_train_df = pd.DataFrame(clinical_train, columns=columns)
    clinical_test_df = pd.DataFrame(clinical_test, columns=columns)

    #print('clinical_train_df shape:', clinical_train_df.shape)
    return clinical_train_df, clinical_test_df



def fit_and_score_features(X, y):
    """
    Scores each feature independently using a univariate CoxnetSurvivalAnalysis.

    Parameters:
    - X: 2D array-like, shape (n_samples, n_features)
    - y: structured array with fields ('event', 'time')

    Returns:
    - scores: array of concordance scores for each feature
    """
    n_features = X.shape[1]
    scores = np.empty(n_features)
    model = CoxnetSurvivalAnalysis(l1_ratio=0.001, alpha_min_ratio=0.001)

    for j in range(n_features):
        Xj = X[:, j:j+1]
        model.fit(Xj, y)
        scores[j] = model.score(Xj, y)

    return scores

def remove_highly_correlated_features(X, y, target_column, threshold=0.9):
    """
    Remove features from X that are highly correlated with each other,
    keeping the one more correlated with the target.

    Parameters:
    - X: pd.DataFrame, feature matrix
    - y: pd.Series or pd.DataFrame, target variable
    - target_column: str, name of the target column
    - threshold: float, correlation threshold for dropping features

    Returns:
    - X_new: pd.DataFrame, with selected features
    """
    # Combine features and target into one DataFrame
    df = pd.concat([X, y], axis=1)

    # Compute correlation matrix
    correlation_matrix = df.corr()

    # Extract upper triangle of the correlation matrix
    upper_triangle = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    )

    # Identify pairs with high correlation
    high_corr_pairs = [
        (col1, col2) for col1 in upper_triangle.columns
        for col2 in upper_triangle.index
        if upper_triangle.at[col2, col1] > threshold
    ]

    # Decide which columns to drop
    columns_to_drop = set()
    for col1, col2 in high_corr_pairs:
        if abs(correlation_matrix.at[target_column, col1]) >= abs(correlation_matrix.at[target_column, col2]):
            columns_to_drop.add(col2)
        else:
            columns_to_drop.add(col1)

    return columns_to_drop

In [3]:
#Load dataset
df = pd.read_csv('data/collagen.csv')

# Filter out missing or zero survival times
#column_name = 'PFS (2022)' #'O.S. (2022)'#
column_name = 'O.S. (2022)'

df = df[df[column_name].notna() & (df[column_name] != 0)].reset_index(drop=True)

# Prepare features and labels
X = df.loc[:, df.columns.str.startswith('col')]
y= pd.DataFrame(df[column_name],columns=[column_name])
X=X.astype(float)
y=y.astype(float)

vital = pd.DataFrame(df['Vital status'].to_list(), columns=['status'])
vital['status'] = vital['status'].replace({'Alive': 1, 'Deceased': 0})
clinical = df[['sex', 'smoking_habit', 'Histological subtype 1_x']]



In [4]:
X.head()

,collagen_1,collagen_2,collagen_3,collagen_4,collagen_5,collagen_6,collagen_7,collagen_8,collagen_9,collagen_10,...,collagen_18,collagen_19,collagen_20,collagen_21,collagen_22,collagen_23,collagen_24,collagen_25,collagen_26,collagen_27
0,1.554794,0.0,2.296140,1.623333,0.0,2.290417,1.679599,0.0,2.292139,1.717588,...,2.295755,1.859709,0.000000,2.293181,1.884007,0.000000,2.288280,1.929493,0.000000,2.294594
1,1.284358,0.0,2.297051,1.393313,0.0,2.292623,1.474194,0.0,2.294564,1.533495,...,2.288909,1.707657,0.000000,2.293531,1.732784,0.000000,2.284521,1.787266,0.000000,2.287021
2,2.011223,0.0,2.296551,2.078783,0.0,2.292801,2.123118,0.0,2.295473,2.153916,...,2.289764,2.208310,2.110577,2.290208,2.224050,2.146151,2.289630,2.233231,2.212584,2.285047
3,0.876276,0.0,2.294742,0.965317,0.0,2.290078,1.044817,0.0,2.292172,1.088438,...,2.292979,1.319557,0.000000,2.292498,1.335192,0.000000,2.296251,1.431000,0.000000,2.292344
4,1.357020,0.0,2.295382,1.447859,0.0,2.297274,1.523586,0.0,2.294368,1.589661,...,2.292441,1.766995,0.000000,2.291823,1.809688,0.000000,2.291379,1.854642,0.000000,2.292968


In [5]:
clinical.head()

,sex,smoking_habit,Histological subtype 1_x
0,Male,Former smoker,Large cell neuroendocrine carcinoma
1,Male,Former smoker,Carcinoid tumor
2,Male,Smoker,Large cell neuroendocrine carcinoma
3,Female,Former smoker,Carcinoid tumor
4,Female,Smoker,Small cell carcinoma


In [6]:

# Initialize results
result_all = [{'seed': 0, 'cv_cindex_rad': 0, 'test_cindex_rad': 0, 'hr_rad': 0,
                'cv_cindex_clin': 0, 'test_cindex_clin': 0, 'hr_clin': 0}]

seeds = [12,60,80,100,799]

In [7]:
# Main loop
for seed in seeds:
    # Split data
    X_train, X_test, y_train, y_test, clin_train, clin_test, vital_train, vital_test = train_test_split(
        X, y, clinical, vital, test_size=0.30, random_state=seed, stratify=vital)

    # Convert to DataFrames with proper columns
    X_train = pd.DataFrame(X_train, columns=X.columns).reset_index(drop=True)
    X_test = pd.DataFrame(X_test, columns=X.columns).reset_index(drop=True)
    clin_train = pd.DataFrame(clin_train, columns=clinical.columns).reset_index(drop=True)
    clin_test = pd.DataFrame(clin_test, columns=clinical.columns).reset_index(drop=True)
    y_train = pd.DataFrame(y_train, columns=y.columns).reset_index(drop=True)
    y_test = pd.DataFrame(y_test, columns=y.columns).reset_index(drop=True)
    vital_train = pd.DataFrame(vital_train, columns=vital.columns).reset_index(drop=True)
    vital_test = pd.DataFrame(vital_test, columns=vital.columns).reset_index(drop=True)

    # Prepare survival labels
    data_y_train = Surv.from_arrays(event=vital_train.status.tolist(), time=y_train[column_name].tolist())
    data_y_test = Surv.from_arrays(event=vital_test.status.tolist(), time=y_test[column_name].tolist())

    # Feature selection
    columns_to_drop = remove_highly_correlated_features(X_train, y_train, target_column=column_name)
    X_train_filtered = X_train.drop(columns=columns_to_drop, errors='ignore')
    X_test_filtered = X_test.drop(columns=columns_to_drop, errors='ignore')

    # Radiomics model pipeline
    pipe = Pipeline([
        ("scaler", StandardScaler()), 
        ("model", CoxnetSurvivalAnalysis()),
    ])

    param_grid = {
        "model__l1_ratio": [ 0.001,0.01,0.02,0.03,0.04, 0.05, 0.06],
        "model__alpha_min_ratio": [0.001,0.002,0.003],
    }

    cv = KFold(n_splits=3, random_state=99, shuffle=True)
    rad_model = GridSearchCV(pipe, param_grid, return_train_score=True, cv=cv, scoring=concordance_index_scorer)
    rad_model.fit(X_train_filtered, data_y_train)

    # Evaluate radiomics model
    best_rad = rad_model.best_estimator_
    rad_cv = rad_model.best_score_
    predicted_risk = best_rad.predict(X_test_filtered)
    rad_test_score = concordance_index_censored(
        vital_test.status.astype(bool).to_numpy(), y_test[column_name].to_numpy(), predicted_risk)
    hr_rad = np.mean(np.exp(best_rad.named_steps["model"].coef_))

    # Feature importance

    x_train_fs = X_train_filtered.copy()
    x_test_fs = X_test_filtered.copy()

    # Clinical model
    clinical_f = 'all_exceptage'
    clinical_train, clinical_test = clinical_data(clin_train, clin_test, clinical_f)

    comb_train = pd.concat([x_train_fs, clinical_train], axis=1)
    comb_test = pd.concat([x_test_fs, clinical_test], axis=1)

    pipe_clin = Pipeline([
        ("scaler", StandardScaler()),
        ("model", CoxnetSurvivalAnalysis()),
    ])

    param_grid_clin = {
        "model__l1_ratio": [0.0001,0.001,0.01,0.02,0.03,0.04,0.05, 0.06],
        "model__alpha_min_ratio": [0.001,0.002,0.003],
    }
    comb_model = GridSearchCV(pipe_clin, param_grid_clin, return_train_score=True, cv=cv, scoring=concordance_index_scorer)
    comb_model.fit(comb_train, data_y_train)

    # Evaluate combined model
    best_comb = comb_model.best_estimator_
    predicted_risk_comb = best_comb.predict(comb_test)
    comb_test_score = concordance_index_censored(
        vital_test.status.astype(bool).to_numpy(), y_test[column_name].to_numpy(), predicted_risk_comb)
    hr_cl = np.mean(np.exp(best_comb.named_steps["model"].coef_))
    comb_cv = comb_model.best_score_

    # Save results
    result_all.append({
        'seed': seed,
        'cv_cindex_rad': rad_cv,
        'test_cindex_rad': rad_test_score[0],
        'hr_rad': hr_rad,
        'cv_cindex_clin': comb_cv,
        'test_cindex_clin': comb_test_score[0],
        'hr_clin': hr_cl
    })


In [8]:
rad_model.best_params_, comb_model.best_params_

({'model__alpha_min_ratio': 0.001, 'model__l1_ratio': 0.001},
 {'model__alpha_min_ratio': 0.003, 'model__l1_ratio': 0.01})

In [9]:
# Save to CSV
df_results = pd.DataFrame(result_all[1:])
#df_results.to_csv('result_all_os.csv', index=False)

In [10]:
df_results

,seed,cv_cindex_rad,test_cindex_rad,hr_rad,cv_cindex_clin,test_cindex_clin,hr_clin
0,12,0.595933,0.720930,1.000569,0.549956,0.744186,1.086863
1,60,0.631746,0.578947,0.990950,0.734921,0.657895,1.008579
2,80,0.639506,0.722222,0.994586,0.787654,0.750000,1.068934
3,100,0.683821,0.521739,0.991494,0.662427,0.543478,0.997136
4,799,0.672742,0.693878,0.991487,0.654844,0.795918,1.003902


In [11]:
np.mean(df_results.cv_cindex_rad), np.std(df_results.cv_cindex_rad),np.mean(df_results.test_cindex_rad),np.std(df_results.test_cindex_rad)

(0.6447495741022661,
 0.031272350793277344,
 0.6475433009313211,
 0.08202626045525349)

In [12]:
confidence_interval(df_results.cv_cindex_rad)
confidence_interval(df_results.test_cindex_rad)

Mean C-index: 0.64, 95% CI: (0.60, 0.69)
Mean C-index: 0.65, 95% CI: (0.53, 0.76)


In [13]:
np.mean(df_results.cv_cindex_clin), np.std(df_results.cv_cindex_clin),np.mean(df_results.test_cindex_clin),np.std(df_results.test_cindex_clin)

(0.6779603308792661,
 0.08054571590428389,
 0.6982954823140475,
 0.08934217481585374)

In [14]:
confidence_interval(df_results.cv_cindex_clin)
confidence_interval(df_results.test_cindex_clin)



Mean C-index: 0.68, 95% CI: (0.57, 0.79)
Mean C-index: 0.70, 95% CI: (0.57, 0.82)
